In [1]:
import json
import os
import glob

import numpy as np 
import pandas as pd
from pandas import Series,DataFrame

In [2]:
train = pd.read_json('arena_data/orig/train.json', typ = 'frame')

In [3]:
train

,tags,id,plylst_title,songs,like_cnt,updt_date
0,"[힐링, 휴식, 밤, 새벽]",147668,To. 힘들고 지친 분들에게,"[663185, 649626, 6855, 188486, 348451, 169945,...",12,2016-06-23 10:06:27.000
1,[팝],50422,130807-7,"[627035, 256438, 603324, 200889, 441319, 21689...",0,2013-08-15 13:17:11.000
2,[뉴에이지],116432,숙면을 위한 슬픈 마음을 달래 줄 피아노,"[129204, 369497, 649743, 344619, 110281, 63266...",23,2015-09-03 16:51:50.000
3,"[하드락, 록스피릿, 댄스]",55076,당신을 하얗게 불태울 곡들,"[677591, 420396, 104934, 119279, 251988, 58850...",1,2017-01-09 15:41:25.000
4,"[힐링, 휴식, 기분전환]",125064,[스피커 필수 / HIPHOP] 듣고 있음 꿀렁꿀렁이고 싶은 힙합음악!,"[704455, 694036, 508043, 154933, 57614, 645195...",715,2016-02-22 12:32:50.000
...,...,...,...,...,...,...
92051,"[90년생, 회상, 추억, 좋은노래, 80년생, 옛날노래]",149690,옛날노래 * 좋은노래 8090년생 노래 모음,"[292099, 513963, 174225, 287212, 140444, 62469...",155,2020-01-15 15:15:45.000
92052,[팝],35004,LOVE 1,"[62596, 359718, 596004, 668790, 291212, 148977...",8,2010-03-23 00:03:00.000
92053,"[여행, 발라드, 기분전환, 사랑]",59765,추억의 2004년 발라드 베스트,"[214372, 145150, 407082, 160552, 102445, 50845...",3,2019-05-15 13:26:07.000
92054,"[소울, 알앤비]",9867,All Music Guide 선정 90s R&B: 1997,"[561958, 397574, 250915, 110345, 426772, 10698...",51,2013-12-24 14:40:01.000


In [4]:
# 플레이리스트 아이디(id)와 수록곡(songs) 추출
plylst_song_map = train[['id', 'songs']]

In [5]:
plylst_song_map

,id,songs
0,147668,"[663185, 649626, 6855, 188486, 348451, 169945,..."
1,50422,"[627035, 256438, 603324, 200889, 441319, 21689..."
2,116432,"[129204, 369497, 649743, 344619, 110281, 63266..."
3,55076,"[677591, 420396, 104934, 119279, 251988, 58850..."
4,125064,"[704455, 694036, 508043, 154933, 57614, 645195..."
...,...,...
92051,149690,"[292099, 513963, 174225, 287212, 140444, 62469..."
92052,35004,"[62596, 359718, 596004, 668790, 291212, 148977..."
92053,59765,"[214372, 145150, 407082, 160552, 102445, 50845..."
92054,9867,"[561958, 397574, 250915, 110345, 426772, 10698..."


In [6]:
# unnest songs
plylst_song_map_unnest = np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))), 
        np.concatenate(plylst_song_map.songs.values)
    )
)

# unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
plylst_song_map['id'] = plylst_song_map['id'].astype(str)
plylst_song_map['songs'] = plylst_song_map['songs'].astype(int)

# unnest 객체 제거
del plylst_song_map_unnest

In [7]:
plylst_song_map.head()

,id,songs
0,147668,663185
1,147668,649626
2,147668,6855
3,147668,188486
4,147668,348451


In [8]:
type(plylst_song_map)

pandas.core.frame.DataFrame

In [9]:
#그냥 song들 만 빼낸거_사실상 무의미
song_matrix = plylst_song_map.drop(['id'], axis='columns')
print(song_matrix)

          songs
0        663185
1        649626
2          6855
3        188486
4        348451
...         ...
4239973  529143
4239974  479142
4239975  239059
4239976  229121
4239977  109325

[4239978 rows x 1 columns]


In [10]:
#유일값 찾기_pandas
song_num=song_matrix['songs'].value_counts()
print(song_num)
song_num=song_num.index
id_num=plylst_song_map['id'].value_counts()
print(id_num)
id_num=id_num.index
#플레이리스트의 id 값: 92056, 플레이리스트 내 song들의 갯수: 549729
print(song_num)

144663    1724
116573    1671
357367    1577
366786    1530
654757    1319
          ... 
388033       1
337270       1
329074       1
390508       1
59360        1
Name: songs, Length: 549729, dtype: int64
109700    200
129149    200
92962     200
78051     200
65767     200
         ... 
14527       1
129499      1
4829        1
16671       1
95559       1
Name: id, Length: 92056, dtype: int64
Int64Index([144663, 116573, 357367, 366786, 654757, 133143, 349492, 675115,
            463173, 396828,
            ...
            424883, 412597, 353662, 396221, 345466, 388033, 337270, 329074,
            390508,  59360],
           dtype='int64', length=549729)


In [11]:
del(song_matrix)

In [12]:
def make_encoding_id_dict(col_unique):
    encoded_dict = {}
    for idx, unique in enumerate(col_unique):
        encoded_dict[unique] = idx
        
    return encoded_dict

In [13]:
#id 와 song을 인코딩해서 딕셔너리 형식으로 바꾸기
encoded_id=make_encoding_id_dict(id_num)
encoded_song=make_encoding_id_dict(song_num)

n_songs = len(song_num)
n_ids = len(id_num)

del(song_num)
del(id_num)

In [14]:
#user-item matrix 만들기(92056, 549729)
user_item_matrix=np.zeros((n_ids,n_songs),dtype=np.int8)

In [15]:
for row in plylst_song_map.itertuples():
    user_item_matrix[encoded_id[row[1]], encoded_song[row[2]]] = 1
    
print(user_item_matrix)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [16]:
from sklearn.metrics import jaccard_similarity_score


In [ ]:
# # from scipy.sparse import dia_matrix
# # import numpy as np

# #user_item_matrix=dia_matrix((549730,92057), dtype=np.int8).toarray()
# # print(d.shape)

# #user-item matrix 만들기
# user_item_matrix=np.zeros((549729,92056),dtype=np.int8)

# #id 별 딕셔너리 만들기
# for key,unique in range(n_songs):
#     print(i, ":", key, n_songs[key])
# print()

# #곡 별 딕셔너리 만들기

# #matrix 셋팅
# for i in range(n_ids+1):
#     user_item_matrix[i+1][0]=song_name[i]
    
# for j in range(n_songs+1):
#     user_item_matrix[0][j+1]=id_num[j]
    
# for i,j in plylst_song_map[['id','songs']]:
#     for m in range(1,len(song_name)):
#         for n in range(1,len(id_name)):
#             user_item_matrix[m][n]=1
    
# #user-item matrix 파일에 저장후 데이터 없애기
# mem_file_name = make_path('user_item_matrix.dat', directory='./data')
# new_np   = make_memmap(mem_file_name,user_item_matrix)
# del user_item_matrix, new_np
# mem_file_name = make_path('user_item_matrix.dat', directory='./data')
# new_np   = read_memmap(mem_file_name)

# # ##파일 저장 후 꺼내서 쓰기
# # # 파일기반 행렬을 만든다
# # mem_file_name = make_path('user_item_matrix.dat', directory='./data')
# # new_np        = make_memmap(mem_file_name,user_item_matrix)

# # print(type(user_item_matrix), type(new_np))

# # del user_item_matrix, new_np

# # # 파일기반 행렬을 읽는다
# # mem_file_name = make_path('user_item_matrix.dat', directory='./data')
# # new_np        = read_memmap(mem_file_name)


# # ##pandas 로 데이터 프레임 만들기
# # user_item_matrix.info(memory_usage='deep')
# # user_item_matrix = pd.DataFrame(np.zeros((549730,92057)))


# # #0,1 기입하기
# # m = 0
# # n = 0

# # for i,j in plylst_song_map[['id','songs']]:
# #     if i==user_item_matrix[m][0]: # id 겹칠때
# #         for line in range(n):
# #             if j==user_item_matrix[m][line]: #노래가 겹칠때
# #                 user_item_matrix[m][line]=1
# #             else: #노래가 겹치지않을때
# #                 n+=1
# #                 user_item_matrix[0][n]=j #노래 목록 추가
# #                 user_item_matrix[m][n]=1
                
# #     elif i==user_item_matrix[(m+1)][0]: #id 안겹칠때
# #         m+=1
# #         user_item_matrix[m][0]=i
# #         for line in range(n):
# #             if j==user_item_matrix[m][line]: #노래가 겹칠때
# #                 user_item_matrix[m][line]=1
# #             else: #노래가 겹치지않을때
# #                 n+=1
# #                 user_item_matrix[0][n]=j #노래 목록 추가
# #                 user_item_matrix[m][n]=1   

In [8]:
import numpy as np
for i in range(10):
    for j in range(10):
        a=[]
        a.append(j)
    np.save('./matrix', a)

In [9]:
new=np.load('./matrix.npy',allow_pickle=True)

In [10]:
print(new)

[9]
